In [1]:
import numpy as np
import pandas as pd

from scipy.stats import gmean

from sklearn.preprocessing import MinMaxScaler

np.set_printoptions(suppress=True)


In [2]:
def internal_process_ahp(filename: str, factor: str):
    # load factor vs factor comparison
    p_crit = pd.read_excel(filename, sheet_name=factor, engine='openpyxl').iloc[:, 1:].values

    # load factor vs alternative comparison and multiply them
    p_alts = None
    for i in range(1, p_crit.shape[0] + 1):
        if p_alts is None:
            p_alts = pd.read_excel(filename, sheet_name=f"F{factor[1]}.{i}").iloc[:, 1:].values
        else:
            p_alts = p_alts * pd.read_excel(filename, sheet_name=f"F{factor[1]}.{i}").iloc[:, 1:].values

    # calculate eigenvalue
    scaler = MinMaxScaler()
    results = scaler.fit_transform(p_alts)

    # calculate mean
    return results.mean(axis=1)

In [3]:
# process all factors
ranks = []
for factor in ["F1", "F2", "F3", "F4", "F5"]:
    ranks.append(internal_process_ahp("ahp1.xlsx", factor))

# run geometric mean score
master_matrix = np.array(ranks)
master_matrix[master_matrix == 0] = 0.0001

# calculate CM
RI = 1.12
CM = master_matrix.prod(axis=1) / master_matrix.mean(axis=1)
# CI = np.abs((CM.mean() - 5) / (5 - 1)) # CM.mean() = lambda_max
CI = CM.mean() # CM.mean() = lambda_max
CR = CI / RI

# solve for ranks
final_rank = master_matrix.prod(axis=0) ** (1/master_matrix.shape[0])

# divide by it's sum to scale it to 1
final_rank = final_rank / final_rank.sum()

print({
    "cr": CR,
    "as1": final_rank[0],
    "as2": final_rank[1],
    "as3": final_rank[2],
    "as4": final_rank[3],
    "as5": final_rank[4],
    "as6": final_rank[5],
})

{'cr': 0.0003365121361194246, 'as1': 0.40587681964425176, 'as2': 0.20872467568601616, 'as3': 0.09829038843688814, 'as4': 0.01478011359513138, 'as5': 0.13767611558214826, 'as6': 0.13465188705556425}


In [5]:
fffs = ["F1", "F2", "F3", "F4", "F5"]
for factor, rank_mat in zip(fffs, ranks):
    print(factor, rank_mat.mean())


F1 0.2393577511040067
F2 0.24989199409670546
F3 0.29166833327907593
F4 0.30378469434696637
F5 0.25663872761184503


In [7]:
factor_weights = np.array(ranks).mean(axis=1)

In [8]:
factor_weights[0]

0.2393577511040067